In [1]:
!pip install geocoder

     |████████████████████████████████| 98 kB 3.1 MB/s 


**Obtaining Neighbourhood data by Web Scraping**

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
from pandas.io.json import json_normalize
import urllib.parse
import matplotlib.pyplot  as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium 
import geocoder
import numpy as np
import geopy.distance

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_areas_of_Vellore"
text = requests.get(url).text

In [ ]:
df = pd.DataFrame()

In [ ]:
soup = BeautifulSoup(text,'lxml')

In [ ]:
neighbourhoods_vellore = []

In [ ]:
for row in soup.findAll('li'):
  neighbourhoods_vellore.append(row.text)
  if row.text == "NANDHIYALAM":
    break

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Kollam"
text = requests.get(url).text

In [ ]:
soup = BeautifulSoup(text,'lxml')

In [ ]:
neighbourhoods_kollam = []

In [ ]:
for row in soup.findAll('li'):
  a = row.text
  if str(a[0]).isdigit():
    continue
  else:
    neighbourhoods_kollam.append(a)
  if a=="Valathungal":
    break

In [ ]:
city = ["Vellore"]*len(neighbourhoods_vellore)+["Kollam"]*len(neighbourhoods_kollam)

In [ ]:
df["Neighbourhood"] = neighbourhoods_vellore+neighbourhoods_kollam
df["City"] = city

In [ ]:
df = df[["Neighbourhood","City"]]

In [ ]:
df

,Neighbourhood,City
0,Thottapalayam,Vellore
1,Gandhi Nagar,Vellore
2,Sathuvachari,Vellore
3,Thorapadi,Vellore
4,Konavattam,Vellore
...,...,...
134,Thattamala,Kollam
135,Thekkevila,Kollam
136,Thekkumbhagam,Kollam
137,Vadakkevila,Kollam


In [ ]:
df.to_csv('/content/drive/MyDrive/Neighbourhood_data.csv')

**Importing Location Data**

In [18]:
df = pd.read_csv('/content/drive/MyDrive/Neighbourhood_data.csv')

In [19]:
df = df[["Neighbourhood","City","Latitude","Longitude"]]

In [20]:
df.dropna(axis=0, inplace=True)

In [21]:
df.isna().sum()

Neighbourhood    0
City             0
Latitude         0
Longitude        0
dtype: int64

**Venue Details using Foursquare API**

In [28]:
address_kollam = 'Kollam, Kerala'
address_vellore = 'Vellore, Tamil Nadu'

geolocator = Nominatim(user_agent="location_explorer")

location_kollam = geolocator.geocode(address_kollam)
location_vellore = geolocator.geocode(address_vellore)

latitude_kollam = location_kollam.latitude
longitude_kollam = location_kollam.longitude

latitude_vellore = location_vellore.latitude
longitude_vellore = location_vellore.longitude

print('The geograpical coordinate of Kollam are {}, {}.'.format(latitude_kollam, longitude_kollam))
print('The geograpical coordinate of Vellore are {}, {}.'.format(latitude_vellore, longitude_vellore))

The geograpical coordinate of Kollam are 8.8879509, 76.5955013.
The geograpical coordinate of Vellore are 12.7948109, 79.0006410968549.


**Mapping the neighbourhoods in Kollam and Vellore**

In [ ]:
# create map of New York using latitude and longitude values
map = folium.Map(location=[latitude_kollam, longitude_kollam], zoom_start=12)

# add markers to map
for i,x in df.iterrows():
    lat = x["Latitude"]
    lng = x["Longitude"]
    city = x["City"]
    neighborhood = x["Neighbourhood"]
    label = '{}, {}'.format(neighborhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

In [ ]:
map = folium.Map(location=[latitude_vellore, longitude_vellore], zoom_start=10)

# add markers to map
for i,x in df.iterrows():
    lat = x["Latitude"]
    lng = x["Longitude"]
    city = x["City"]
    neighborhood = x["Neighbourhood"]
    label = '{}, {}'.format(neighborhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

**Testing Foursquare API on a data sample**

In [ ]:
df["Neighbourhood"][0]

'Thottapalayam'

In [ ]:
lat_neighborhood = df["Latitude"][0]
long_neighborhood = df["Longitude"][0]
name_neighborhood = df["Neighbourhood"][0]
radius = 1500
print("Latitude : {} Longitude : {} Name : {}".format(lat_neighborhood,long_neighborhood,name_neighborhood))

Latitude : 12.9327 Longitude : 79.1424 Name : Thottapalayam


In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat_neighborhood, 
    long_neighborhood, 
    radius, 
    LIMIT)

In [ ]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60fadc74ace6051c01bd69e8'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-60f50a15b6a0e627c8b1b5d8-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
          'suffix': '.png'},
         'id': '52f2ab2ebcbc57f1066b8b27',
         'name': 'Mattress Store',
         'pluralName': 'Mattress Stores',
         'primary': True,
         'shortName': 'Mattress Store'}],
       'id': '60f50a15b6a0e627c8b1b5d8',
       'location': {'address': 'Kollam Trade Centre, Puthenchantha,',
        'cc': 'IN',
        'city': 'Varkala',
        'country': 'India',
        'distance': 943,
        'formattedAddress': ['Kollam Trade Centre, Puthenchantha,',
         'Varkala 695141',
         'Kerala',
         'India'],
        'labeledL

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Sleepwell,Mattress Store,8.86448,76.6327


In [ ]:
venues_list = []
for _,x in df.iterrows():
  neighbourhood = x["Neighbourhood"]
  city = x["City"]
  latitude = x["Latitude"]
  longitude = x["Longitude"]
  radius = 2000
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
  results = requests.get(url).json()["response"]['groups'][0]['items']
  venues_list.append([(
            neighbourhood,
            city, 
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighbourhood',
                         'City', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']


In [ ]:
print("There are {} different categories".format(len(nearby_venues["Venue Category"].unique())))

There are 81 different categories


If we examine these categories we can see categories like



*   Restaurant
*   Vegetarian/Vegan Restaurant
*   Kerala Restaurant
*   Pizza place
*   Fast food Restaurant
*   Indian Restaurant

and many more like these which we can possibly combine.










In [ ]:
nearby_venues.to_csv('/content/drive/MyDrive/nearby_venues.csv')

In [3]:
nearby_venues = pd.read_csv('/content/drive/MyDrive/nearby_venues.csv')

In [4]:
nearby_venues["Venue Category"].value_counts()

Indian Restaurant     169
Restaurant             60
Clothing Store         50
Movie Theater          44
Beach                  43
                     ... 
Tennis Court            1
Supermarket             1
Moving Target           1
Athletics & Sports      1
Campground              1
Name: Venue Category, Length: 70, dtype: int64

**One hot encode the nearby venues data to give as input to the K means algorithm**

In [52]:
encoded_input = pd.get_dummies(nearby_venues[["Venue Category"]], prefix = "", prefix_sep = "")

In [53]:
encoded_input.insert(0, 'Neighbourhood', nearby_venues[["Neighbourhood"]])

In [54]:
encoded_input.head()

,Neighbourhood,ATM,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Bakery,Bar,Basketball Court,Beach,Boat or Ferry,Bookstore,Boutique,Bus Station,Bus Stop,Cafe,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Fast Food Restaurant,Fish Market,Flower Shop,Furniture / Home Store,Gym,Historic Site,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Market,Mattress Store,Memorial Site,Men's Store,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Park,Pharmacy,Pizza Place,Platform,Playground,Resort,Restaurant,Seafood Restaurant,Snack Place,Spa,Sporting Goods Shop,Supermarket,Tennis Court,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Volleyball Court
0,Thottapalayam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Thottapalayam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Thottapalayam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Thottapalayam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Thottapalayam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
encoded_input = encoded_input.groupby('Neighbourhood').mean().reset_index()

In [56]:
encoded_input_data = encoded_input.drop(labels=["Neighbourhood"],axis=1)

**Fit K Means with cluster = 7**

In [57]:
kmeans = KMeans(n_clusters=7).fit(encoded_input_data)

In [58]:
encoded_input.insert(1, 'Cluster Labels', kmeans.labels_)

In [59]:
encoded_input

,Neighbourhood,Cluster Labels,ATM,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Bakery,Bar,Basketball Court,Beach,Boat or Ferry,Bookstore,Boutique,Bus Station,Bus Stop,Cafe,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Fast Food Restaurant,Fish Market,Flower Shop,Furniture / Home Store,Gym,Historic Site,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Market,Mattress Store,Memorial Site,Men's Store,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Park,Pharmacy,Pizza Place,Platform,Playground,Resort,Restaurant,Seafood Restaurant,Snack Place,Spa,Sporting Goods Shop,Supermarket,Tennis Court,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Volleyball Court
0,A.M.Puram (Alamelumangapuram),4,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.083333,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.083333,0.0,0.083333,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
1,Abdullapuram,1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.333333,0.0,0.000000,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
2,Akkolil,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.2,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.0,0.000000,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.20,0.0,0.0,0.0
3,Allapuram,6,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
4,Ambur,4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.25,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Velpadi,6,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.142857,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.00,0.0,0.0,0.0
112,Virupatchipuram,6,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
113,Walajapet,1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.

In [60]:
final = df.join(encoded_input.set_index('Neighbourhood'), on='Neighbourhood')

In [61]:
final.head()

,Neighbourhood,City,Latitude,Longitude,Cluster Labels,ATM,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Bakery,Bar,Basketball Court,Beach,Boat or Ferry,Bookstore,Boutique,Bus Station,Bus Stop,Cafe,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Fast Food Restaurant,Fish Market,Flower Shop,Furniture / Home Store,Gym,Historic Site,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Market,Mattress Store,Memorial Site,Men's Store,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Park,Pharmacy,Pizza Place,Platform,Playground,Resort,Restaurant,Seafood Restaurant,Snack Place,Spa,Sporting Goods Shop,Supermarket,Tennis Court,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Volleyball Court
0,Thottapalayam,Vellore,12.9327,79.1424,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.142857,0.0,0.0,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Gandhi Nagar,Vellore,12.9460,79.1492,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.166667,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Sathuvachari,Vellore,12.9399,79.1681,1.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.000000,0.200000,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0
3,Thorapadi,Vellore,12.8915,79.1238,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.4,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Konavattam,Vellore,12.9231,79.1114,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
final["Cluster Labels"].isna().sum()

8

In [63]:
final = final[final["Cluster Labels"].notna()]

**Visualizing the clusters inside Kollam**

In [64]:
# create map
map_clusters = folium.Map(location=[latitude_kollam, longitude_kollam], zoom_start=11)
clusters = 17
# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final['Latitude'], final['Longitude'], final['Neighbourhood'], final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(int(cluster)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**The Neighbourhood my friend is looking to move out of is Kilikollur, Kollam**

In [65]:
final[final["Neighbourhood"]=="Kilikollur"]

,Neighbourhood,City,Latitude,Longitude,Cluster Labels,ATM,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Bakery,Bar,Basketball Court,Beach,Boat or Ferry,Bookstore,Boutique,Bus Station,Bus Stop,Cafe,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Fast Food Restaurant,Fish Market,Flower Shop,Furniture / Home Store,Gym,Historic Site,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Market,Mattress Store,Memorial Site,Men's Store,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Park,Pharmacy,Pizza Place,Platform,Playground,Resort,Restaurant,Seafood Restaurant,Snack Place,Spa,Sporting Goods Shop,Supermarket,Tennis Court,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Volleyball Court
110,Kilikollur,Kollam,8.9179,76.6326,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0


**The cluster label of kilikollur is 0.0. Now we need to find the neighbourhoods in vellore in the same cluster**

In [79]:
final_vellore = final[final["City"]=="Vellore"]
suggestions = final_vellore[final_vellore["Cluster Labels"]==1.0]
len(suggestions["Neighbourhood"].unique())

14

**We have successfully obtained 14 possible locations to live that are similar to Kilikollur, Kollam**





In [68]:
#Visualizing the suggestions

# create map
map_clusters = folium.Map(location=[latitude_vellore, longitude_vellore], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(suggestions['Latitude'], suggestions['Longitude'], suggestions['Neighbourhood'], suggestions['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(int(cluster)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Now that we havve obtained the places similar to kilikollur in Vellore we need to optimize the solution and find the places closest to college that my friend can relocate to**

In [81]:
coords_VIT = (12.9717,79.1594)

In [82]:
distance = []
for _,x in suggestions.iterrows():
  coords_sugg = (x["Latitude"],x["Longitude"])

  distance.append(geopy.distance.vincenty(coords_sugg,coords_VIT).km)

In [83]:
suggestions.insert(5,"Distance to VIT", distance)

In [ ]:
suggestions.sort_values(by="Distance to VIT", axis=0, inplace=True)

In [85]:
suggestions.head(6)

,Neighbourhood,City,Latitude,Longitude,Cluster Labels,Distance to VIT,ATM,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Bakery,Bar,Basketball Court,Beach,Boat or Ferry,Bookstore,Boutique,Bus Station,Bus Stop,Cafe,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Fast Food Restaurant,Fish Market,Flower Shop,Furniture / Home Store,Gym,Historic Site,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Market,Mattress Store,Memorial Site,Men's Store,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Park,Pharmacy,Pizza Place,Platform,Playground,Resort,Restaurant,Seafood Restaurant,Snack Place,Spa,Sporting Goods Shop,Supermarket,Tennis Court,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Volleyball Court
5,Dharapadavedu,Vellore,12.9695,79.1386,1.0,2.269835,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.214286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.071429,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.071429,0.0
17,Katpadi,Vellore,12.9734,79.1369,1.0,2.448409,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.090909,0.0,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.090909,0.0
1,Gandhi Nagar,Vellore,12.9460,79.1492,1.0,3.051000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
36,Vallalar,Vellore,12.9443,79.1633,1.0,3.060659,0.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.000000,0.0,0.000000,0.0
2,Sathuvachari,Vellore,12.9399,79.1681,1.0,3.642485,0.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.000000,0.0,0.000000,0.0
39,Kazhinjur,Vellore,12.9527,79.1280,1.0,4.003187,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400000,0.0,0.0,0.0,0.000000,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


**These are the top 4 suggestions for my friend to move to based on similarity to previous residence area and distance to location of study**